# Hodina 14: CSP - Constraint Satisfaction Problems 🧩

## Přehled lekce

V této hodině se naučíme:
- Pochopit a modelovat CSP problémy
- Implementovat backtracking s neuronovými heuristikami
- Použít transformery pro CSP solving
- Vizualizovat proces řešení CSP
- Vytvořit interaktivní CSP řešič

---

## 1. Nastavení prostředí a instalace knihoven

In [ ]:
# Instalace potřebných knihoven
!pip install torch torchvision transformers gradio networkx matplotlib numpy
!pip install plotly ipywidgets python-constraint ortools

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
import networkx as nx
from collections import defaultdict, deque
import time
from IPython.display import HTML, display, clear_output
import plotly.graph_objects as go
import gradio as gr
from transformers import GPT2Model, GPT2Config
import json
import random
from typing import List, Dict, Set, Tuple, Optional
from constraint import Problem, AllDifferentConstraint
from ortools.sat.python import cp_model

# Nastavení zobrazení
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
torch.manual_seed(42)

  Preparing metadata (setup.py) ...   Preparing metadata (setup.py) ... -done
one
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/20.2 MB ? eta -:--:--Downloading ortools-9.14.6206-cp313-cp313-macosx_11_0_arm64.whl (20.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 51.8 MB/s  0:00:00m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 51.8 MB/s  0:00:00
  DEPRECATION: Building 'python-constraint' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'python-constraint'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'python-constraint' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future versio

## 2. Co je CSP? - Teorie a základy 📚

CSP se skládá z:
- **Proměnných** (Variables)
- **Domén** (Domains) - možné hodnoty
- **Omezení** (Constraints) - pravidla

In [ ]:
# Základní CSP framework
class CSPProblem:
    def __init__(self):
        self.variables = []
        self.domains = {}
        self.constraints = []
        self.assignment = {}
        self.assignment_history = []
    
    def add_variable(self, var, domain):
        """Přidá proměnnou s doménou"""
        self.variables.append(var)
        self.domains[var] = list(domain)
    
    def add_constraint(self, constraint):
        """Přidá omezení"""
        self.constraints.append(constraint)
    
    def is_consistent(self, var, value, assignment):
        """Kontroluje, zda je přiřazení konzistentní"""
        assignment[var] = value
        for constraint in self.constraints:
            if not constraint.is_satisfied(assignment):
                del assignment[var]
                return False
        del assignment[var]
        return True
    
    def is_complete(self, assignment):
        """Kontroluje, zda je přiřazení kompletní"""
        return len(assignment) == len(self.variables)

# Základní typy omezení
class Constraint:
    def __init__(self, variables):
        self.variables = variables
    
    def is_satisfied(self, assignment):
        raise NotImplementedError

class AllDifferentConstraint(Constraint):
    """Všechny proměnné musí mít různé hodnoty"""
    def is_satisfied(self, assignment):
        values = []
        for var in self.variables:
            if var in assignment:
                if assignment[var] in values:
                    return False
                values.append(assignment[var])
        return True

class BinaryConstraint(Constraint):
    """Binární omezení mezi dvěma proměnnými"""
    def __init__(self, var1, var2, relation):
        super().__init__([var1, var2])
        self.relation = relation
    
    def is_satisfied(self, assignment):
        if self.variables[0] in assignment and self.variables[1] in assignment:
            return self.relation(assignment[self.variables[0]], 
                               assignment[self.variables[1]])
        return True

# Příklad: N-Queens problém
class NQueensProblem(CSPProblem):
    def __init__(self, n=8):
        super().__init__()
        self.n = n
        
        # Proměnné: řádky, hodnoty: sloupce
        for i in range(n):
            self.add_variable(f'Q{i}', range(n))
        
        # Omezení: žádné dvě dámy se nesmí ohrozit
        for i in range(n):
            for j in range(i+1, n):
                # Různé sloupce
                self.add_constraint(
                    BinaryConstraint(f'Q{i}', f'Q{j}', lambda x, y: x != y)
                )
                # Různé diagonály
                self.add_constraint(
                    BinaryConstraint(f'Q{i}', f'Q{j}', 
                                   lambda x, y, i=i, j=j: abs(x-y) != abs(i-j))
                )
    
    def visualize(self, assignment=None):
        """Vizualizuje šachovnici s dámami"""
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Šachovnice
        for i in range(self.n):
            for j in range(self.n):
                color = 'white' if (i + j) % 2 == 0 else 'lightgray'
                rect = patches.Rectangle((j, i), 1, 1, facecolor=color)
                ax.add_patch(rect)
        
        # Dámy
        if assignment:
            for var, col in assignment.items():
                row = int(var[1:])
                circle = patches.Circle((col + 0.5, row + 0.5), 0.3, 
                                      facecolor='red', edgecolor='darkred')
                ax.add_patch(circle)
                ax.text(col + 0.5, row + 0.5, '♛', fontsize=20, 
                       ha='center', va='center', color='white')
        
        ax.set_xlim(0, self.n)
        ax.set_ylim(0, self.n)
        ax.set_aspect('equal')
        ax.invert_yaxis()
        ax.set_title(f'{self.n}-Queens Problem')
        plt.show()

# Demonstrace
print("👑 N-Queens problém jako CSP:")
queens = NQueensProblem(4)
print(f"Proměnné: {queens.variables}")
print(f"Domény: {queens.domains}")
print(f"Počet omezení: {len(queens.constraints)}")

# Ukázka vizualizace
queens.visualize()

## 3. Backtracking algoritmus s vizualizací 🔄

Implementujeme základní backtracking s různými heuristikami.

In [ ]:
# Backtracking solver s vizualizací
class BacktrackingSolver:
    def __init__(self, csp):
        self.csp = csp
        self.nodes_explored = 0
        self.backtrack_count = 0
        self.solution_found = False
        self.search_tree = []
    
    def solve(self, assignment=None, visualize=False):
        """Hlavní backtracking algoritmus"""
        if assignment is None:
            assignment = {}
        
        # Uložení kroku pro vizualizaci
        self.search_tree.append({
            'assignment': dict(assignment),
            'complete': self.csp.is_complete(assignment),
            'nodes_explored': self.nodes_explored
        })
        
        if self.csp.is_complete(assignment):
            self.solution_found = True
            return assignment
        
        # Výběr neprřazené proměnné
        var = self.select_unassigned_variable(assignment)
        
        # Zkouška hodnot
        for value in self.order_domain_values(var, assignment):
            self.nodes_explored += 1
            
            if self.csp.is_consistent(var, value, assignment):
                assignment[var] = value
                
                if visualize:
                    self.visualize_step(assignment)
                
                result = self.solve(assignment, visualize)
                if result is not None:
                    return result
                
                del assignment[var]
                self.backtrack_count += 1
        
        return None
    
    def select_unassigned_variable(self, assignment):
        """MRV (Minimum Remaining Values) heuristika"""
        unassigned = [v for v in self.csp.variables if v not in assignment]
        
        # MRV: vybere proměnnou s nejmenším počtem zbývajících hodnot
        min_values = float('inf')
        best_var = unassigned[0]
        
        for var in unassigned:
            count = 0
            for value in self.csp.domains[var]:
                if self.csp.is_consistent(var, value, assignment):
                    count += 1
            
            if count < min_values:
                min_values = count
                best_var = var
        
        return best_var
    
    def order_domain_values(self, var, assignment):
        """LCV (Least Constraining Value) heuristika"""
        values = []
        
        for value in self.csp.domains[var]:
            # Spočítá kolik hodnot zbude sousedům
            constraining_count = 0
            assignment[var] = value
            
            for other_var in self.csp.variables:
                if other_var != var and other_var not in assignment:
                    for other_value in self.csp.domains[other_var]:
                        if not self.csp.is_consistent(other_var, other_value, assignment):
                            constraining_count += 1
            
            del assignment[var]
            values.append((value, constraining_count))
        
        # Seřadí podle nejméně omezujících hodnot
        values.sort(key=lambda x: x[1])
        return [v[0] for v in values]
    
    def visualize_step(self, assignment):
        """Vizualizuje krok řešení"""
        clear_output(wait=True)
        if hasattr(self.csp, 'visualize'):
            self.csp.visualize(assignment)
        time.sleep(0.5)
    
    def get_statistics(self):
        """Vrátí statistiky řešení"""
        return {
            'nodes_explored': self.nodes_explored,
            'backtrack_count': self.backtrack_count,
            'solution_found': self.solution_found,
            'search_tree_size': len(self.search_tree)
        }

# Test backtracking solveru
print("🔄 Řešení 4-Queens pomocí backtracking:")
queens_4 = NQueensProblem(4)
solver = BacktrackingSolver(queens_4)

solution = solver.solve()
if solution:
    print(f"\n✅ Řešení nalezeno: {solution}")
    queens_4.visualize(solution)
    
    stats = solver.get_statistics()
    print(f"\n📊 Statistiky:")
    for key, value in stats.items():
        print(f"  {key}: {value}")
else:
    print("❌ Řešení nenalezeno")

## 4. Neuronové sítě pro CSP solving 🧠

Použijeme neuronové sítě pro učení heuristik a predikci řešení.

In [ ]:
# Neuronová síť pro predikci hodnot proměnných v CSP
class NeuralCSPSolver(nn.Module):
    def __init__(self, max_vars=20, max_domain_size=20, hidden_dim=256):
        super(NeuralCSPSolver, self).__init__()
        
        # Encoder pro proměnné a domény
        self.var_embedding = nn.Embedding(max_vars, hidden_dim)
        self.domain_embedding = nn.Embedding(max_domain_size, hidden_dim)
        
        # Transformer pro zachycení vztahů mezi proměnnými
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=hidden_dim,
                nhead=8,
                dim_feedforward=hidden_dim * 4,
                batch_first=True
            ),
            num_layers=4
        )
        
        # Dekodér pro predikci hodnot
        self.value_predictor = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, max_domain_size)
        )
        
        # Hodnotící síť pro odhad obtížnosti
        self.difficulty_estimator = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, var_indices, domain_masks, constraint_matrix):
        # var_indices: [batch, n_vars]
        # domain_masks: [batch, n_vars, max_domain]
        # constraint_matrix: [batch, n_vars, n_vars]
        
        # Embed proměnné
        var_embeds = self.var_embedding(var_indices)
        
        # Přidání informace o doménách
        domain_info = domain_masks.float().mean(dim=2, keepdim=True)
        var_embeds = var_embeds * domain_info
        
        # Transformer processing
        transformed = self.transformer(var_embeds)
        
        # Predikce hodnot pro každou proměnnou
        value_logits = self.value_predictor(transformed)
        
        # Odhad obtížnosti
        difficulty = self.difficulty_estimator(transformed.mean(dim=1))
        
        return value_logits, difficulty

# Generátor trénovacích dat pro CSP
class CSPDataGenerator:
    def __init__(self, problem_types=['n_queens', 'graph_coloring', 'sudoku']):
        self.problem_types = problem_types
    
    def generate_n_queens_data(self, n=8, n_samples=100):
        """Generuje data z N-Queens problémů"""
        data = []
        
        for _ in range(n_samples):
            # Vytvoření problému
            problem = NQueensProblem(n)
            solver = BacktrackingSolver(problem)
            solution = solver.solve()
            
            if solution:
                # Převod na tensory
                var_indices = torch.arange(n)
                domain_masks = torch.ones(n, n)  # Všechny pozice možné
                
                # Jednoduchá reprezentace omezení
                constraint_matrix = torch.zeros(n, n)
                for i in range(n):
                    for j in range(n):
                        if i != j:
                            constraint_matrix[i, j] = 1
                
                # Cílové hodnoty
                target = torch.zeros(n, n)
                for var, val in solution.items():
                    idx = int(var[1:])
                    target[idx, val] = 1
                
                data.append({
                    'var_indices': var_indices,
                    'domain_masks': domain_masks,
                    'constraint_matrix': constraint_matrix,
                    'target': target,
                    'difficulty': solver.nodes_explored / (n * n)
                })
        
        return data
    
    def generate_graph_coloring_data(self, n_nodes=10, n_colors=3, n_samples=100):
        """Generuje data z problémů barvení grafů"""
        data = []
        
        for _ in range(n_samples):
            # Náhodný graf
            G = nx.erdos_renyi_graph(n_nodes, 0.3)
            
            # CSP problém
            problem = CSPProblem()
            for node in G.nodes():
                problem.add_variable(f'node_{node}', range(n_colors))
            
            # Omezení - sousední uzly různé barvy
            for edge in G.edges():
                problem.add_constraint(
                    BinaryConstraint(f'node_{edge[0]}', f'node_{edge[1]}', 
                                   lambda x, y: x != y)
                )
            
            solver = BacktrackingSolver(problem)
            solution = solver.solve()
            
            if solution:
                # Převod na tensory
                var_indices = torch.arange(n_nodes)
                domain_masks = torch.ones(n_nodes, n_colors)
                
                # Matice sousednosti jako constraint matrix
                constraint_matrix = torch.FloatTensor(
                    nx.adjacency_matrix(G).todense()
                )
                
                # Cílové hodnoty
                target = torch.zeros(n_nodes, n_colors)
                for var, val in solution.items():
                    idx = int(var.split('_')[1])
                    target[idx, val] = 1
                
                data.append({
                    'var_indices': var_indices,
                    'domain_masks': domain_masks,
                    'constraint_matrix': constraint_matrix,
                    'target': target,
                    'difficulty': solver.nodes_explored / (n_nodes * n_colors)
                })
        
        return data

# Trénování neuronového CSP solveru
print("🧠 Trénování neuronového CSP solveru...")

# Generování dat
generator = CSPDataGenerator()
train_data = generator.generate_n_queens_data(n=6, n_samples=50)
print(f"Vygenerováno {len(train_data)} trénovacích vzorků")

# Model a trénování
model = NeuralCSPSolver(max_vars=20, max_domain_size=20)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

# Jednoduchý trénovací loop
losses = []
for epoch in range(50):
    epoch_loss = 0
    
    for sample in train_data:
        optimizer.zero_grad()
        
        # Forward pass
        value_logits, difficulty = model(
            sample['var_indices'].unsqueeze(0),
            sample['domain_masks'].unsqueeze(0),
            sample['constraint_matrix'].unsqueeze(0)
        )
        
        # Loss
        target = sample['target'].unsqueeze(0)
        loss = criterion(value_logits, target)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_data)
    losses.append(avg_loss)
    
    if epoch % 10 == 0:
        print(f"Epocha {epoch}: Loss = {avg_loss:.4f}")

# Vizualizace trénování
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title('Průběh trénování neuronového CSP solveru')
plt.xlabel('Epocha')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

## 5. Příklad: Sudoku solver s vizualizací 🔢

Implementujeme kompletní Sudoku solver jako CSP.

In [ ]:
# Sudoku jako CSP
class SudokuProblem(CSPProblem):
    def __init__(self, puzzle):
        super().__init__()
        self.puzzle = puzzle
        self.size = 9
        
        # Vytvoření proměnných a domén
        for i in range(9):
            for j in range(9):
                var = f'cell_{i}_{j}'
                if puzzle[i][j] == 0:
                    self.add_variable(var, range(1, 10))
                else:
                    self.add_variable(var, [puzzle[i][j]])
        
        # Omezení pro řádky
        for i in range(9):
            row_vars = [f'cell_{i}_{j}' for j in range(9)]
            self.add_constraint(AllDifferentConstraint(row_vars))
        
        # Omezení pro sloupce
        for j in range(9):
            col_vars = [f'cell_{i}_{j}' for i in range(9)]
            self.add_constraint(AllDifferentConstraint(col_vars))
        
        # Omezení pro 3x3 bloky
        for box_row in range(3):
            for box_col in range(3):
                box_vars = []
                for i in range(3):
                    for j in range(3):
                        var = f'cell_{box_row*3+i}_{box_col*3+j}'
                        box_vars.append(var)
                self.add_constraint(AllDifferentConstraint(box_vars))
    
    def visualize(self, assignment=None, highlight_cell=None):
        """Vizualizuje Sudoku mřížku"""
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Mřížka
        for i in range(10):
            lw = 2 if i % 3 == 0 else 1
            ax.axhline(i, color='black', linewidth=lw)
            ax.axvline(i, color='black', linewidth=lw)
        
        # Čísla
        for i in range(9):
            for j in range(9):
                var = f'cell_{i}_{j}'
                
                # Barva pozadí
                if highlight_cell and highlight_cell == (i, j):
                    rect = patches.Rectangle((j, 8-i), 1, 1, 
                                           facecolor='yellow', alpha=0.5)
                    ax.add_patch(rect)
                
                # Hodnota
                if assignment and var in assignment:
                    value = assignment[var]
                    color = 'blue' if self.puzzle[i][j] == 0 else 'black'
                    ax.text(j + 0.5, 8.5 - i, str(value), 
                           fontsize=16, ha='center', va='center', 
                           color=color, weight='bold')
                elif self.puzzle[i][j] != 0:
                    ax.text(j + 0.5, 8.5 - i, str(self.puzzle[i][j]), 
                           fontsize=16, ha='center', va='center', 
                           color='black', weight='bold')
        
        ax.set_xlim(0, 9)
        ax.set_ylim(0, 9)
        ax.set_aspect('equal')
        ax.axis('off')
        ax.set_title('Sudoku Solver', fontsize=20)
        plt.tight_layout()
        plt.show()

# Vylepšený backtracking pro Sudoku
class SudokuSolver(BacktrackingSolver):
    def __init__(self, sudoku_problem):
        super().__init__(sudoku_problem)
        self.inference_count = 0
    
    def solve_with_inference(self, assignment=None, use_ac3=True):
        """Backtracking s Arc Consistency (AC-3)"""
        if assignment is None:
            assignment = {}
        
        if self.csp.is_complete(assignment):
            return assignment
        
        # AC-3 inference
        if use_ac3:
            inference_result = self.ac3_inference(assignment)
            if not inference_result:
                return None
        
        var = self.select_unassigned_variable(assignment)
        
        for value in self.order_domain_values(var, assignment):
            if self.csp.is_consistent(var, value, assignment):
                assignment[var] = value
                
                result = self.solve_with_inference(assignment, use_ac3)
                if result is not None:
                    return result
                
                del assignment[var]
        
        return None
    
    def ac3_inference(self, assignment):
        """Arc Consistency algoritmus"""
        queue = []
        
        # Inicializace fronty všemi oblouky
        for constraint in self.csp.constraints:
            if len(constraint.variables) == 2:
                queue.append((constraint.variables[0], constraint.variables[1]))
                queue.append((constraint.variables[1], constraint.variables[0]))
        
        while queue:
            xi, xj = queue.pop(0)
            
            if self.revise(xi, xj, assignment):
                if len(self.get_consistent_values(xi, assignment)) == 0:
                    return False
                
                # Přidání sousedů do fronty
                for constraint in self.csp.constraints:
                    if xi in constraint.variables:
                        for xk in constraint.variables:
                            if xk != xi and xk != xj:
                                queue.append((xk, xi))
        
        return True
    
    def revise(self, xi, xj, assignment):
        """Redukce domény xi vzhledem k xj"""
        revised = False
        xi_values = self.get_consistent_values(xi, assignment)
        
        for x in xi_values:
            consistent = False
            for y in self.get_consistent_values(xj, assignment):
                test_assignment = dict(assignment)
                test_assignment[xi] = x
                test_assignment[xj] = y
                
                # Kontrola konzistence
                valid = True
                for constraint in self.csp.constraints:
                    if xi in constraint.variables and xj in constraint.variables:
                        if not constraint.is_satisfied(test_assignment):
                            valid = False
                            break
                
                if valid:
                    consistent = True
                    break
            
            if not consistent:
                # Odstranění hodnoty z domény
                # V této implementaci to simulujeme
                revised = True
        
        return revised
    
    def get_consistent_values(self, var, assignment):
        """Vrátí konzistentní hodnoty pro proměnnou"""
        if var in assignment:
            return [assignment[var]]
        
        values = []
        for value in self.csp.domains[var]:
            if self.csp.is_consistent(var, value, assignment):
                values.append(value)
        return values

# Test Sudoku solveru
print("🔢 Sudoku Solver:")

# Příklad Sudoku puzzle (0 = prázdné)
puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

sudoku = SudokuProblem(puzzle)
print("Původní Sudoku:")
sudoku.visualize()

# Řešení
solver = SudokuSolver(sudoku)
start_time = time.time()
solution = solver.solve_with_inference()
solve_time = time.time() - start_time

if solution:
    print(f"\n✅ Řešení nalezeno za {solve_time:.3f} sekund!")
    sudoku.visualize(solution)
else:
    print("❌ Řešení nenalezeno")

## 6. Transformer pro CSP solving 🤖

Implementujeme pokročilý transformer model pro řešení CSP.

In [ ]:
# Transformer CSP Solver
class TransformerCSPSolver(nn.Module):
    def __init__(self, d_model=512, n_heads=8, n_layers=6, max_vars=100, max_domain=100):
        super(TransformerCSPSolver, self).__init__()
        
        # Embeddings
        self.var_embedding = nn.Embedding(max_vars, d_model)
        self.value_embedding = nn.Embedding(max_domain, d_model)
        self.constraint_embedding = nn.Linear(max_vars, d_model)
        
        # Pozicový encoding
        self.pos_encoding = self._create_positional_encoding(max_vars, d_model)
        
        # Transformer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # Výstupní hlavy
        self.value_head = nn.Linear(d_model, max_domain)
        self.confidence_head = nn.Linear(d_model, 1)
        
        # Attention pro interpretaci
        self.attention_weights = None
    
    def _create_positional_encoding(self, max_len, d_model):
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                           -(np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        return nn.Parameter(pe.unsqueeze(0), requires_grad=False)
    
    def forward(self, var_ids, current_assignment, constraint_graph):
        # var_ids: [batch, n_vars]
        # current_assignment: [batch, n_vars] (-1 for unassigned)
        # constraint_graph: [batch, n_vars, n_vars]
        
        batch_size, n_vars = var_ids.shape
        
        # Embed proměnné
        var_embeds = self.var_embedding(var_ids)
        
        # Přidání informace o aktuálním přiřazení
        assigned_mask = (current_assignment != -1).float().unsqueeze(-1)
        var_embeds = var_embeds * (1 - assigned_mask * 0.5)
        
        # Přidání constraint informace
        constraint_embeds = self.constraint_embedding(constraint_graph)
        var_embeds = var_embeds + constraint_embeds.mean(dim=2)
        
        # Pozicový encoding
        var_embeds = var_embeds + self.pos_encoding[:, :n_vars, :]
        
        # Transformer processing
        transformed = self.transformer(var_embeds)
        
        # Predikce hodnot
        value_logits = self.value_head(transformed)
        confidence = self.confidence_head(transformed).sigmoid()
        
        return value_logits, confidence

# Hybridní solver kombinující neuronové sítě a klasické metody
class HybridCSPSolver:
    def __init__(self, neural_model, traditional_solver):
        self.neural_model = neural_model
        self.traditional_solver = traditional_solver
        self.use_neural_guidance = True
    
    def solve(self, csp_problem, max_neural_attempts=10):
        """Řeší CSP kombinací neuronového a klasického přístupu"""
        
        # Nejprve zkusí neuronový model
        if self.use_neural_guidance:
            neural_solution = self.neural_solve(csp_problem, max_neural_attempts)
            if neural_solution:
                return neural_solution, 'neural'
        
        # Pokud neuronový model selže, použije klasický solver
        traditional_solution = self.traditional_solver.solve()
        return traditional_solution, 'traditional'
    
    def neural_solve(self, csp_problem, max_attempts):
        """Pokusí se vyřešit CSP pomocí neuronového modelu"""
        
        for attempt in range(max_attempts):
            assignment = {}
            
            # Převod CSP na tensory
            var_ids = torch.LongTensor([i for i in range(len(csp_problem.variables))])
            current_assignment = torch.full((1, len(csp_problem.variables)), -1)
            constraint_graph = self._build_constraint_graph(csp_problem)
            
            # Postupné přiřazování
            for _ in range(len(csp_problem.variables)):
                # Neuronová predikce
                with torch.no_grad():
                    value_logits, confidence = self.neural_model(
                        var_ids.unsqueeze(0),
                        current_assignment,
                        constraint_graph.unsqueeze(0)
                    )
                
                # Výběr proměnné s nejvyšší jistotou
                unassigned = [i for i, var in enumerate(csp_problem.variables) 
                            if var not in assignment]
                
                if not unassigned:
                    break
                
                var_confidences = confidence[0, unassigned]
                best_var_idx = unassigned[var_confidences.argmax().item()]
                best_var = csp_problem.variables[best_var_idx]
                
                # Výběr hodnoty
                var_logits = value_logits[0, best_var_idx]
                domain_indices = torch.LongTensor(csp_problem.domains[best_var])
                valid_logits = var_logits[domain_indices]
                
                best_value_idx = valid_logits.argmax().item()
                best_value = csp_problem.domains[best_var][best_value_idx]
                
                # Kontrola konzistence
                if csp_problem.is_consistent(best_var, best_value, assignment):
                    assignment[best_var] = best_value
                    current_assignment[0, best_var_idx] = best_value
                else:
                    break
            
            # Kontrola úplnosti řešení
            if csp_problem.is_complete(assignment):
                return assignment
        
        return None
    
    def _build_constraint_graph(self, csp_problem):
        """Vytvoří matici reprezentující constraint graf"""
        n_vars = len(csp_problem.variables)
        graph = torch.zeros(n_vars, n_vars)
        
        var_to_idx = {var: i for i, var in enumerate(csp_problem.variables)}
        
        for constraint in csp_problem.constraints:
            for i, var1 in enumerate(constraint.variables):
                for j, var2 in enumerate(constraint.variables):
                    if i != j and var1 in var_to_idx and var2 in var_to_idx:
                        graph[var_to_idx[var1], var_to_idx[var2]] = 1
        
        return graph

# Demonstrace transformer CSP solveru
print("🤖 Transformer CSP Solver:")

# Vytvoření modelu
transformer_solver = TransformerCSPSolver(d_model=256, n_heads=8, n_layers=4)
print(f"Model má {sum(p.numel() for p in transformer_solver.parameters())} parametrů")

# Test na malém problému
small_queens = NQueensProblem(4)
traditional_solver = BacktrackingSolver(small_queens)
hybrid_solver = HybridCSPSolver(transformer_solver, traditional_solver)

# Řešení
solution, method = hybrid_solver.solve(small_queens)
if solution:
    print(f"\n✅ Řešení nalezeno metodou: {method}")
    print(f"Řešení: {solution}")

## 7. Interaktivní CSP Explorer 🎮

Vytvoříme Gradio aplikaci pro exploraci CSP problémů.

In [ ]:
# Interaktivní CSP Explorer
class CSPExplorer:
    def __init__(self):
        self.current_problem = None
        self.solver = None
        self.solution = None
    
    def create_problem(self, problem_type, size):
        """Vytvoří CSP problém"""
        if problem_type == "N-Queens":
            self.current_problem = NQueensProblem(size)
        elif problem_type == "Sudoku":
            # Jednoduchý příklad
            puzzle = [[0]*9 for _ in range(9)]
            # Přidání několika čísel
            puzzle[0][0] = 5
            puzzle[0][1] = 3
            puzzle[1][0] = 6
            self.current_problem = SudokuProblem(puzzle)
        elif problem_type == "Graph Coloring":
            # Vytvoření grafu
            self.current_problem = self._create_graph_coloring_problem(size)
        
        return self._visualize_problem()
    
    def _create_graph_coloring_problem(self, n_nodes):
        """Vytvoří problém barvení grafu"""
        problem = CSPProblem()
        
        # Náhodný graf
        G = nx.erdos_renyi_graph(n_nodes, 0.3)
        
        # Proměnné a domény
        colors = ['červená', 'modrá', 'zelená', 'žlutá']
        for node in G.nodes():
            problem.add_variable(f'node_{node}', colors)
        
        # Omezení
        for edge in G.edges():
            problem.add_constraint(
                BinaryConstraint(f'node_{edge[0]}', f'node_{edge[1]}', 
                               lambda x, y: x != y)
            )
        
        problem.graph = G  # Uložení pro vizualizaci
        return problem
    
    def _visualize_problem(self):
        """Vizualizuje aktuální problém"""
        if self.current_problem is None:
            return None
        
        if hasattr(self.current_problem, 'visualize'):
            self.current_problem.visualize()
            plt.savefig('problem_viz.png', dpi=150, bbox_inches='tight')
            plt.close()
            return 'problem_viz.png'
        elif hasattr(self.current_problem, 'graph'):
            # Vizualizace grafu
            plt.figure(figsize=(8, 8))
            nx.draw(self.current_problem.graph, with_labels=True, 
                   node_color='lightblue', node_size=1000, font_size=16)
            plt.title('Graf pro barvení')
            plt.savefig('problem_viz.png', dpi=150, bbox_inches='tight')
            plt.close()
            return 'problem_viz.png'
        
        return None
    
    def solve_problem(self, algorithm, use_heuristics):
        """Řeší aktuální problém"""
        if self.current_problem is None:
            return None, "Nejprve vytvořte problém!"
        
        # Výběr solveru
        if algorithm == "Backtracking":
            if hasattr(self.current_problem, 'puzzle'):
                self.solver = SudokuSolver(self.current_problem)
            else:
                self.solver = BacktrackingSolver(self.current_problem)
        
        # Řešení
        start_time = time.time()
        
        if hasattr(self.solver, 'solve_with_inference') and use_heuristics:
            self.solution = self.solver.solve_with_inference()
        else:
            self.solution = self.solver.solve()
        
        solve_time = time.time() - start_time
        
        # Vizualizace řešení
        if self.solution:
            if hasattr(self.current_problem, 'visualize'):
                self.current_problem.visualize(self.solution)
                plt.savefig('solution_viz.png', dpi=150, bbox_inches='tight')
                plt.close()
                viz_file = 'solution_viz.png'
            elif hasattr(self.current_problem, 'graph'):
                # Vizualizace obarveného grafu
                plt.figure(figsize=(8, 8))
                
                # Mapování barev
                color_map = {'červená': 'red', 'modrá': 'blue', 
                           'zelená': 'green', 'žlutá': 'yellow'}
                node_colors = []
                for node in self.current_problem.graph.nodes():
                    var = f'node_{node}'
                    if var in self.solution:
                        node_colors.append(color_map[self.solution[var]])
                    else:
                        node_colors.append('gray')
                
                nx.draw(self.current_problem.graph, with_labels=True,
                       node_color=node_colors, node_size=1000, font_size=16)
                plt.title('Obarvený graf')
                plt.savefig('solution_viz.png', dpi=150, bbox_inches='tight')
                plt.close()
                viz_file = 'solution_viz.png'
            else:
                viz_file = None
        else:
            viz_file = None
        
        # Statistiky
        stats = self.solver.get_statistics()
        
        result_text = f"""
        📊 Výsledky řešení:
        - Algoritmus: {algorithm}
        - Čas řešení: {solve_time:.3f} sekund
        - Prozkoumaných uzlů: {stats['nodes_explored']}
        - Počet backtrack: {stats['backtrack_count']}
        - Řešení nalezeno: {'Ano' if self.solution else 'Ne'}
        """
        
        return viz_file, result_text
    
    def analyze_complexity(self):
        """Analyzuje složitost problému"""
        if self.current_problem is None:
            return None, "Nejprve vytvořte problém!"
        
        # Analýza
        n_vars = len(self.current_problem.variables)
        n_constraints = len(self.current_problem.constraints)
        
        avg_domain_size = np.mean([len(domain) for domain in self.current_problem.domains.values()])
        
        # Graf constraint density
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Distribuce velikostí domén
        domain_sizes = [len(domain) for domain in self.current_problem.domains.values()]
        ax1.hist(domain_sizes, bins=20, color='skyblue', edgecolor='black')
        ax1.set_xlabel('Velikost domény')
        ax1.set_ylabel('Počet proměnných')
        ax1.set_title('Distribuce velikostí domén')
        
        # Constraint graph
        constraint_graph = nx.Graph()
        for var in self.current_problem.variables:
            constraint_graph.add_node(var)
        
        for constraint in self.current_problem.constraints:
            if len(constraint.variables) == 2:
                constraint_graph.add_edge(constraint.variables[0], 
                                        constraint.variables[1])
        
        degrees = [d for n, d in constraint_graph.degree()]
        ax2.hist(degrees, bins=20, color='lightcoral', edgecolor='black')
        ax2.set_xlabel('Stupeň uzlu')
        ax2.set_ylabel('Počet uzlů')
        ax2.set_title('Distribuce stupňů v constraint grafu')
        
        plt.tight_layout()
        plt.savefig('complexity_analysis.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        analysis_text = f"""
        📈 Analýza složitosti:
        - Počet proměnných: {n_vars}
        - Počet omezení: {n_constraints}
        - Průměrná velikost domény: {avg_domain_size:.1f}
        - Maximální velikost stavového prostoru: {avg_domain_size**n_vars:.2e}
        - Constraint density: {n_constraints / (n_vars * (n_vars - 1) / 2):.2f}
        """
        
        return 'complexity_analysis.png', analysis_text

# Vytvoření Gradio rozhraní
explorer = CSPExplorer()

with gr.Blocks(title="CSP Explorer") as demo:
    gr.Markdown("# 🧩 CSP Explorer - Interaktivní průzkum CSP problémů")
    
    with gr.Tab("Vytvoření problému"):
        with gr.Row():
            with gr.Column():
                problem_type = gr.Dropdown(
                    choices=["N-Queens", "Sudoku", "Graph Coloring"],
                    value="N-Queens",
                    label="Typ problému"
                )
                size_slider = gr.Slider(
                    minimum=4, maximum=12, value=8, step=1,
                    label="Velikost problému"
                )
                create_btn = gr.Button("Vytvořit problém", variant="primary")
            
            with gr.Column():
                problem_viz = gr.Image(label="Vizualizace problému")
        
        create_btn.click(
            fn=explorer.create_problem,
            inputs=[problem_type, size_slider],
            outputs=problem_viz
        )
    
    with gr.Tab("Řešení problému"):
        with gr.Row():
            with gr.Column():
                algorithm_select = gr.Dropdown(
                    choices=["Backtracking", "AC-3", "Neural-Guided"],
                    value="Backtracking",
                    label="Algoritmus"
                )
                use_heuristics = gr.Checkbox(
                    label="Použít heuristiky (MRV, LCV)",
                    value=True
                )
                solve_btn = gr.Button("Vyřešit", variant="primary")
            
            with gr.Column():
                solution_viz = gr.Image(label="Řešení")
                solution_stats = gr.Textbox(label="Statistiky", lines=8)
        
        solve_btn.click(
            fn=explorer.solve_problem,
            inputs=[algorithm_select, use_heuristics],
            outputs=[solution_viz, solution_stats]
        )
    
    with gr.Tab("Analýza složitosti"):
        analyze_btn = gr.Button("Analyzovat složitost", variant="primary")
        complexity_viz = gr.Image(label="Analýza")
        complexity_text = gr.Textbox(label="Výsledky analýzy", lines=8)
        
        analyze_btn.click(
            fn=explorer.analyze_complexity,
            inputs=[],
            outputs=[complexity_viz, complexity_text]
        )
    
    gr.Markdown(
        """
        ### 📝 Návod:
        1. **Vytvoření problému**: Vyberte typ a velikost CSP problému
        2. **Řešení**: Zvolte algoritmus a spusťte řešení
        3. **Analýza**: Prozkoumejte složitost problému
        
        ### 🎯 Typy problémů:
        - **N-Queens**: Umístění N dam na šachovnici
        - **Sudoku**: Klasické Sudoku puzzle
        - **Graph Coloring**: Obarvení grafu minimálním počtem barev
        """
    )

# Spuštění aplikace
print("🚀 Spouštím CSP Explorer...")
demo.launch(share=True)

## 8. Praktická cvičení 📝

In [ ]:
# Cvičení 1: Implementujte Map Coloring problém
class MapColoringProblem(CSPProblem):
    """
    TODO: Implementujte problém barvení mapy
    - Státy jako proměnné
    - Barvy jako domény
    - Sousední státy musí mít různé barvy
    """
    def __init__(self, countries, neighbors, colors):
        super().__init__()
        # Váš kód zde
        pass

# Cvičení 2: Forward Checking
def forward_checking(csp, var, value, assignment, domains):
    """
    TODO: Implementujte forward checking
    - Odstraňte nekonzistentní hodnoty z domén
    - Vraťte True pokud jsou všechny domény neprázdné
    """
    # Váš kód zde
    pass

# Cvičení 3: Neuronová heuristika pro výběr proměnné
class NeuralVariableSelector(nn.Module):
    """
    TODO: Vytvořte neuronovou síť pro výběr nejlepší proměnné
    Vstup: stav CSP (proměnné, domény, omezení)
    Výstup: skóre pro každou neprřazenou proměnnou
    """
    def __init__(self):
        super(NeuralVariableSelector, self).__init__()
        # Váš kód zde
        pass

# Cvičení 4: Constraint Learning
class ConstraintLearner:
    """
    TODO: Implementujte systém pro učení omezení z příkladů
    - Vstup: pozitivní a negativní příklady řešení
    - Výstup: naučená omezení
    """
    def __init__(self):
        # Váš kód zde
        pass
    
    def learn_from_examples(self, positive_examples, negative_examples):
        # Váš kód zde
        pass

print("📚 Cvičení připravena!")
print("\nTipy:")
print("- Pro cvičení 1: Použijte AllDifferentConstraint pro sousedy")
print("- Pro cvičení 2: Iterujte přes omezení a aktualizujte domény")
print("- Pro cvičení 3: Použijte GNN pro zachycení struktury grafu")
print("- Pro cvičení 4: Zvažte decision trees nebo rule learning")

## 9. Shrnutí a klíčové koncepty 🎓

### Co jsme se naučili:

1. **CSP komponenty**
   - Variables (proměnné)
   - Domains (domény)
   - Constraints (omezení)

2. **Algoritmy řešení**
   - Backtracking
   - Forward checking
   - Arc consistency (AC-3)
   - Heuristiky: MRV, LCV

3. **Neuronové přístupy**
   - Učení heuristik
   - Transformer architektury
   - Hybridní řešení

4. **Praktické aplikace**
   - N-Queens
   - Sudoku
   - Graph coloring
   - Scheduling

### Klíčové techniky:
- **MRV**: Minimum Remaining Values
- **Degree heuristic**: Nejvíce omezená proměnná
- **LCV**: Least Constraining Value
- **Constraint propagation**: Šíření omezení

### Další kroky:
- V další hodině: **Pokročilé CSP techniky a propagace omezení**
- Naučíme se globální omezení a symetrie

In [ ]:
# Závěrečné shrnutí
print("🎉 Gratulujeme! Dokončili jste hodinu o CSP!")
print("\n📊 Vaše pokroky:")
print("✅ Pochopení CSP konceptů")
print("✅ Implementace backtracking algoritmu")
print("✅ Použití neuronových sítí pro CSP")
print("✅ Vytvoření interaktivního CSP exploreru")
print("✅ Řešení praktických problémů")

# Srovnání přístupů
print("\n📋 Srovnání přístupů k CSP:")
comparison = {
    'Metoda': ['Backtracking', 'FC', 'AC-3', 'Neural'],
    'Kompletnost': ['Ano', 'Ano', 'Ne*', 'Ne'],
    'Optimální': ['Ano**', 'Ano**', 'N/A', 'Ne'],
    'Rychlost': ['Pomalá', 'Střední', 'Rychlá', 'Velmi rychlá***']
}

import pandas as pd
df = pd.DataFrame(comparison)
print(df.to_string(index=False))
print("\n* AC-3 je preprocessing technika")
print("** Pro nalezení jakéhokoliv řešení")
print("*** Pokud je dobře natrénovaná")

print("\n🚀 Připraveni na pokročilé CSP techniky v další hodině!")

# Hodina 14 — Heuristiky a A* (ELI10)

Krátce a jednoduše: heuristika je tip nebo odhad, kolik ještě zbývá do cíle. A* je chytrý algoritmus, který při hledání cesty kombinuje skutečně ujetou vzdálenost (g) a heuristický odhad (h) a dává přednost cestám s nejmenším součtem f = g + h.

Co si zapamatuj: heuristika by měla být "admisibilní" (nikdy nepřecenit skutečnou vzdálenost), aby A* našel nejkratší cestu. V praxi použijeme např. Manhattanovu vzdálenost na mřížce.

Níže je jednoduchý, ale úplně funkční příklad A* na mřížce — spusť ho v Colabu nebo Jupyteru a vyzkoušej si ho.

In [ ]:
import heapq

# A* na mřížce (0 = průchozí, 1 = překážka)

def heuristic(a, b):
    # Manhattanova vzdálenost
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def astar(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    open_set = []
    heapq.heappush(open_set, (0 + heuristic(start, goal), 0, start, None))
    came_from = {}
    g_score = {start: 0}

    while open_set:
        f, g, current, parent = heapq.heappop(open_set)
        if current in came_from:
            continue
        came_from[current] = parent
        if current == goal:
            # reconstruct path
            path = []
            node = current
            while node is not None:
                path.append(node)
                node = came_from[node]
            return list(reversed(path))

        r, c = current
        for dr, dc in [(1,0),(-1,0),(0,1),(0,-1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == 0:
                neighbor = (nr, nc)
                tentative_g = g + 1
                if neighbor in g_score and tentative_g >= g_score[neighbor]:
                    continue
                g_score[neighbor] = tentative_g
                heapq.heappush(open_set, (tentative_g + heuristic(neighbor, goal), tentative_g, neighbor, current))

    return None  # cesta neexistuje

# Malý testovací grid
grid = [
    [0,0,0,0,0],
    [0,1,1,1,0],
    [0,0,0,1,0],
    [0,1,0,0,0],
    [0,0,0,0,0],
]
start = (0,0)
goal = (4,4)
path = astar(grid, start, goal)
print("Nalezená cesta:", path)
assert path is not None, "A* nenašel cestu, přestože existuje"
# ověřme, že cesta začíná a končí správně
assert path[0] == start and path[-1] == goal
# ověřme délku (nejkratší vzdálenost v tomto gridu by měla být 8)
assert len(path)-1 == 8
print("A* testy prošly.")

Cvičení a úkoly

1. Změň heuristiku na Euclidovu vzdálenost a porovnej chování s Manhattanem (zjisti, zda stále dostaneš nejkratší cestu).
2. Přidej váhy (různé náklady pohybu) do gridu — uprav A* tak, aby fungoval s váhami místo jednotkových kroků.
3. Implementuj omezení (např. maximální délku cesty) a uprav algoritmus tak, aby vracel nejlepší řešení do daného limitu.

Další rozšíření: Použij A* na reálné mapě (např. OpenStreetMap) nebo porovnej A* s Dijkstrou na různých typech map.